# Day 12 - Steering a ship

In [1]:
using System.IO;

var input = File.ReadAllLines(@".\input.txt")
    .Select(x => (instr: x[0], dist: Convert.ToInt32(x.Substring(1))))
    .ToArray();

input

index,Item1,Item2
0,F,35
1,L,90
2,S,5
3,F,4
4,R,90
5,F,46
6,W,3
7,N,1
8,L,90
9,F,13


## 12.1 Solution

In [1]:
var itinerary = new List<(char op, int dist, int x, int y)>();
var curPos = (x: 0, y: 0);
var faceDir = 0;

foreach(var op in input)
{
    faceDir = op.instr switch {
        'L' => (faceDir - op.dist + 360) % 360,
        'R' => (faceDir + op.dist) % 360,
        _ => faceDir
    };

    curPos = op.instr switch {
        'L' or 'R' => curPos,
        'F' => faceDir switch {
            0 => (x: curPos.x + op.dist, y: curPos.y), // east
            90 => (x: curPos.x, y: curPos.y + op.dist), // south
            180 => (x: curPos.x - op.dist, y: curPos.y), // west
            270 => (x: curPos.x, y: curPos.y - op.dist), // north
            _ => throw new Exception("Diagonals not supported")
        },
        'E' => (x: curPos.x + op.dist, y: curPos.y),
        'S' => (x: curPos.x, y: curPos.y + op.dist),
        'W' => (x: curPos.x - op.dist, y: curPos.y),
        'N' => (x: curPos.x, y: curPos.y - op.dist),
        _ => throw new Exception("Not supported")
    };

    itinerary.Add((op.instr, op.dist, curPos.x, curPos.y));
}

curPos.x + curPos.y

590

## 12.2 Solution

In [1]:
var itinerary = new List<(char op, int dist, int x, int y, int z, int i)>();
var curWP = (x: 10, y: 1);
var curPos = (x: 0, y: 0);
var faceDir = 0;

foreach(var op in input)
{
    curWP = op.instr switch {
        'F' => curWP,
        'L' => (
            x: (int)(curWP.x * Math.Cos(-1*op.dist/180*Math.PI) - curWP.y * Math.Sin(-1*op.dist/180*Math.PI)), 
            y: (int)(curWP.x * Math.Sin(-1*op.dist/180*Math.PI) + curWP.y * Math.Cos(-1*op.dist/180*Math.PI))
        ),
        'R' => (
            x: (int)(curWP.x * Math.Cos(op.dist/180*Math.PI) - curWP.y * Math.Sin(op.dist/180*Math.PI)), 
            y: (int)(curWP.x * Math.Sin(op.dist/180*Math.PI) + curWP.y * Math.Cos(op.dist/180*Math.PI))
        ),
        'E' => (x: curWP.x + op.dist, y: curWP.y),
        'S' => (x: curWP.x, y: curWP.y + op.dist),
        'W' => (x: curWP.x - op.dist, y: curWP.y),
        'N' => (x: curWP.x, y: curWP.y - op.dist),
        _ => throw new Exception("Not supported")
    };

    curPos = op.instr switch {
        'F' => (x: curPos.x + curWP.x * op.dist, y: curPos.y + curWP.y * op.dist),
        _ => curPos
    };

    itinerary.Add((op.instr, op.dist, curWP.x, curWP.y, curPos.x, curPos.y));
}

Console.WriteLine(curPos.x + curPos.y);

itinerary

//curWP.x + curWP.y

69485


index,Item1,Item2,Item3,Item4,Item5,Item6
0,F,35,10,1,350,35
1,L,90,10,10,350,35
2,S,5,10,15,350,35
3,F,4,10,15,390,95
4,R,90,10,15,390,95
5,F,46,10,15,850,785
6,W,3,7,15,850,785
7,N,1,7,14,850,785
8,L,90,7,7,850,785
9,F,13,7,7,941,876


In [1]:
var curWP = (x: 10, y: 1);
var angle = -90.0;
var transformed = (
    x: (curWP.x * Math.Cos(angle/180.0*Math.PI) - curWP.y * Math.Sin(angle/180.0*Math.PI)), 
    y: (curWP.x * Math.Sin(angle/180.0*Math.PI) + curWP.y * Math.Cos(angle/180.0*Math.PI))
);

(curWP.x * Math.Cos(angle/180.0*Math.PI)).ToString("n5")
// transformed

0,00000